# Cheatsheet on "How to use Spark SQL"


   In this notebook, I try to make readable and easy to use reference for [SQL](https://en.wikipedia.org/wiki/SQL) users aiming to explain how to do similar action in [Spark SQL](https://spark.apache.org/docs/2.3.1/api/sql/).  
   
|Spark     | Impala|
|:------:   | :------:|
|![Spark](https://spark.apache.org/docs/2.3.1/img/spark-logo-hd.png)| ![Impala](https://impala.apache.org/img/impala-logo.png)|


<br>

I tried to summerize SQL basic commands and to point out some Spark SQL exclusive statements and compare them with Impala SQL.
<br> 

## Common basic SQL command
[W3school](https://www.w3schools.com/sql/default.asp) provided a good tutorial on SQL statements. Here I try to give some useful examples.

#### General `SELECT` statement:



```sql
SELECT column1, column2, ...
FROM table_name
WHERE condition;
```

Example:
```sql
SELECT purchase_date, product, customer_id
FROM my_table
WHERE purchase_date > 18700
```

#### Aggregate statement (`Group by`)
Useful aggregate functions: `count()`, `sum()`, `avg()`
```sql
SELECT column_name(s)
FROM table_name
WHERE condition
GROUP BY column_name(s)
ORDER BY column_name(s) ASC|DESC;;
```

Example:
```sql
SELECT purchase_date, product, customer_id, sum(price)
FROM my_table
WHERE purchase_date > 18700
GROUP BY purchase_date, product, customer_id
ORDER BY purchase_date ASC
```

#### Select TOP

|  TOP X   |   LIMIT X |
|:----------|:-----------|
|`SELECT TOP number\|percent column_name(s)` <br> `FROM table_name` <br> `WHERE condition;`|`SELECT column_name(s)` <br> `FROM table_name` <br> `WHERE condition` <br> `LIMIT number;`|
|**SQL Server**<br> **Sybase** | **MySQL**<br> **Impala**<br> **Spark** |

## Impala Spark Differences

### Group By Alias

**Impala:**

```sql
SELECT from_unixtime(floor((begin_time + 7*1800)/3600)*3600, 'dd-MMM-yyyy HH:mm') as qdatehour, sum(price)
from my_table
where product = 'Milk'
group by qdatehour
```

**Spark:**

```sql
SELECT from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH') as qdatehour, sum(price)
from my_table
where product = 'Milk'
group by from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH')
```


Notes:

    1- Spark automatically converts unix time to local time, but Impala needs conversion to local time first (+3.5 hours)
    
    2- in Spark the alias cannot be used in `group by`, but in Impala it is possible.




### Order By Alias

**Impala:**

```sql
SELECT from_unixtime(floor((begin_time + 7*1800)/3600)*3600, 'dd-MMM-yyyy HH:mm') as qdatehour, sum(price)
from my_table
where product = 'Milk'
group by qdatehour
order by qdatehour
```

**Spark:**

```sql
SELECT from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH') as qdatehour, sum(price)
from my_table
where product = 'Milk'
group by from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH')
order by qdatehour
```

#### Using `UNION` and `ORDER BY` together:

```sql
SELECT from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH') as qdatehour, sum(price)
from my_table1
where product = 'Milk'
group by from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH')

UNION ALL

SELECT from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH') as qdatehour, sum(price)
from my_table2
where product = 'Milk'
group by from_unixtime(BEGIN_TIME, 'dd-MMM-yyyy HH')

order by qdatehour

```




Notes:

    1- Spark automatically converts unix time to local time, but Impala needs conversion to local time first (+3.5 hours)
    
    2- In Spark the alias cannot be used in `GROUP BY`, but in Impala it is possible.
    
    3- When using `UNION` place `ORDER BY` at the end of script
    
    4- If alias used, `ORDER BY` of multiple statements must be the alias.


### Minutes Granularity

15 minutes example:

**Impala:**

```sql
SELECT from_unixtime(floor((begin_time + 7*1800)/900)*900, 'dd-MMM-yyyy HH:mm') as qdatehour, sum(price)
from my_table
where product = 'Milk'
group by qdatehour
```

**Spark:**

```sql
SELECT from_unixtime(floor(begin_time/900)*900, 'dd-MMM-yyyy HH:mm') as qdatehour, sum(price)
from my_table
where product = 'Milk'
group by from_unixtime(floor(begin_time/900)*900, 'dd-MMM-yyyy HH:mm')
```


Notes:

    1- Spark automatically converts unix time to local time, but Impala needs conversion to local time first (+3.5 hours)
    

### String Functions

substring

**Impala:**
```sql
strleft(purcahse_date, 4) as purchase_year
```

**Spark:**
```sql
substr(purcahse_date, 1, 8) as purchase_year


SELECT CONCAT("SQL ", "Tutorial ", "is ", "fun!") AS ConcatenatedString;
```

### Date Functions



**Spark:**
```sql
SELECT CURRENT_DATE();
select CURRENT_TIMESTAMP() 
select DATEDIFF(CURRENT_DATE(),cast('2021-01-5' as date)) 
SELECT DATE_ADD("2021-01-15",  3);
```

### Other Functions

**Impala and Spark:**
```sql
SELECT ROUND(135.375, 2);
```

### Other Functions Comparison

**Impala:**
```sql
SELECT ISNULL('Salam', 500);
```


**Spark:**
```sql
SELECT IFNULL('Salam', 500);

SELECT case when 'Salam' is null THEN 500 ELSE 'Salam' END 

```

Example:

```sql
SELECT (case when sum(price) is null THEN 0 ELSE sum(price) END) as Total_price
from my_table
where product = 'Milk'

SELECT (IFNULL(sum(price), 0) ) as Total_price
from my_table
where product = 'Milk'
```